# TEST Set Evaluation - Reporting

In [ ]:
!pip install torch
!pip install torchmetrics
from torch.nn import functional as F
import torch
!pip install transformers
#from transformers import RobertaTokenizerFast
from transformers import RobertaForSequenceClassification, Trainer
from google.colab import drive
!pip3 install pickle5
import pickle5 as pickle
import os
## needed for this task in particular
from heapq import nlargest
from torch.utils.data import DataLoader
from torchmetrics import RetrievalMAP

     |████████████████████████████████| 276kB 8.0MB/s 
     |████████████████████████████████| 2.3MB 7.8MB/s 
     |████████████████████████████████| 901kB 46.8MB/s 
     |████████████████████████████████| 3.3MB 43.0MB/s 
     |████████████████████████████████| 133kB 8.3MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219282 sha256=928bf2f25aa189d2e687a6004f0adbe9bc43325b5abf319cb89bd44e67273ca0
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5


In [ ]:
def save_as_pickle(obj, filename):
    """
    save an object in a pickle file dump
    :param obj: object to dump
    :param filename: target file
    :return:
    """
    directory = os.path.dirname(filename)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    with open(filename, 'wb') as file:
        pickle.dump(obj, file, protocol=pickle.HIGHEST_PROTOCOL)


def load_pickle(filename):
    """
    load an object from a given pickle file
    :param filename: source file
    :return: loaded object
    """
    with open(filename, 'rb') as file:
        return pickle.load(file)

In [ ]:
drive.mount('/content/drive', force_remount=True)  # use force_remount=True param after upload of new data

Mounted at /content/drive


In [ ]:
# file and folder names
#ir_project_drive_folder = "Master/2 - FSS 2021/Information Retrieval/IR Projekt"  # TODO: geht das für alle?
ir_project_drive_folder = "IR Projekt"  # TODO: geht das für alle?
full_ir_project_drive_folder = "/content/drive/My Drive/{}/".format(ir_project_drive_folder)

# full_ir_project_drive_folder = '../data/wikipedia'
raw_folder = full_ir_project_drive_folder + 'data/wikipedia/raw'
preprocessed_folder = full_ir_project_drive_folder + 'data/wikipedia/no-pron/preprocessed'
bm25_folder = full_ir_project_drive_folder + 'data/wikipedia/no-pron/bm25'
# raw qa filenames
qa_wikipedia_test_without_answers_filename = raw_folder + '/qa/wikipedia-test-without-answers.json'

# preprocessed evidence files
wikipedia_evidence_file = raw_folder + '/wikipedia_evidence_dict.pkl'
preprocessed_wikipedia_evidence_file = preprocessed_folder + '/preprocessed_wikipedia_evidence_dict.pkl'

# preprocessed qa files
preprocessed_qa_wikipedia_test_without_answers_filename = preprocessed_folder + '/qa/wikipedia-test-without-answers.pkl'

# retrieved BM25
bm25_retrieval_test_without_answers_filename = bm25_folder + '/retrieval_wiki_test_docs_scores.pkl'
bm25_retrieved_docs_TP_FP = full_ir_project_drive_folder + "data/wikipedia/Neural_Reranking_Test/bm25_tp_fp.pkl"

nr_models_folder = full_ir_project_drive_folder + 'saved_models/neural_retrieval'
# first cycle (random irrelevant) - approximate
model_path_roberta_first_5000_15000_wd_2e = nr_models_folder + '/roberta_first_5000_15000_wd_2e'

# second cycle (random hard) - 5000 - 15000
model_path_roberta_second_randhard_5000_15000_wd_2e = nr_models_folder + '/roberta_second_randhard_5000_15000_wd_2e'


## encoding 
RoBERTa_encoding_path_100 = full_ir_project_drive_folder + 'data/wikipedia/Neural_Reranking_Test/RoBERTa/encoding100test.pkl'

## reranked - OUTPUT
bm25_rr_100_roberta_5000_15000_wd_1 = full_ir_project_drive_folder + 'data/wikipedia/Neural_Reranking_Test/RoBERTa/First_5000_15000.pkl'
bm25_rr_100_roberta_5000_15000_wd_2_rh = full_ir_project_drive_folder + 'data/wikipedia/Neural_Reranking_Test/RoBERTa/Second_5000_15000_randhard.pkl'


## Data Loading and Prep

### Test Set

In [ ]:
def rerank_topX(retrieved, X, topX):
  for query in retrieved:
    interim={}
    interim = dict(retrieved[query])
    topX[query] = nlargest(X, interim, key = interim.get)

In [ ]:
bm25_retrieval_test_without_answers = load_pickle(bm25_retrieval_test_without_answers_filename)

test_top_50_bm25={}

rerank_topX(bm25_retrieval_test_without_answers, 50, test_top_50_bm25)
print(len(bm25_retrieval_test_without_answers))
bm25_retrieval_test_without_answers= None

len(test_top_50_bm25)

7701


7701

### Other


In [ ]:
preprocessed_qa_wikipedia_test_without_answers = load_pickle(os.path.join(preprocessed_qa_wikipedia_test_without_answers_filename))

documents_dict = load_pickle(wikipedia_evidence_file)

## Tokenization & MAP Prep


In [ ]:
# tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

In [ ]:
# ## Tokenize
# docs = []
# queries = []
# q_id = []  
# doc_id = []

# for qa_pair in preprocessed_qa_wikipedia_test_without_answers[:100]:
#   query_id = qa_pair['QuestionId']
#   for d_id in test_top_50_bm25[query_id]:
#     document = documents_dict[d_id]
#     docs.append(document)
#     queries.append(qa_pair['Question'])
#     q_id.append(qa_pair['QuestionId'])
#     doc_id.append(d_id)
# encoding = tokenizer(queries, docs, truncation=True, padding="max_length", return_tensors='pt') #, return_token_type_ids=False


In [ ]:
# save_as_pickle(encoding, RoBERTa_encoding_path_100)

## First Cycle: 5000 - 15000 RoBertA

In [ ]:
encodings = load_pickle(RoBERTa_encoding_path_100)

In [ ]:
model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_first_5000_15000_wd_2e)
output_dir = bm25_rr_100_roberta_5000_15000_wd_1

In [ ]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings, boundaries=None):
    if boundaries is not None:
      self.encodings = {key: val[boundaries[0]: boundaries[1]] for key, val in encodings.items()}
    else:
      self.encodings = encodings

  def __getitem__(self, idx):
      item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
      return item

  def __len__(self):
      return len(self.encodings['input_ids'])

trainer = Trainer(
    model = model
)

def get_preds(dataset):
  prediction_output = trainer.predict(dataset)
  predictions = prediction_output.predictions
  preds = torch.tensor(predictions)
  res = F.softmax(preds, dim = 1)
  results = res[:,1]
  del predictions
  del prediction_output
  del preds
  del res
  torch.cuda.empty_cache()
  return results


In [ ]:
sub_dataset = Dataset(encodings)
preds_array = get_preds(sub_dataset)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


In [ ]:
print(preds_array)
print(torch.max(preds_array))
print(torch.min(preds_array))

tensor([0.9987, 0.9987, 0.9987,  ..., 0.0012, 0.9250, 0.0027])
tensor(0.9987)
tensor(0.0009)


In [ ]:
## connect predictions w/ query_id and docs
i= 0
retrieval = {}
qids = []
for qa_pair in preprocessed_qa_wikipedia_test_without_answers[:100]:
  query_id = qa_pair['QuestionId']
  for d_id in test_top_50_bm25[query_id]:
    if query_id in qids:
      retrieval[query_id].append((d_id, float(preds_array[i])))
    else: 
      retrieval[query_id]= [(d_id, float(preds_array[i]))]
      qids.append(query_id)
    i += 1
    
print(len(retrieval))

for key in retrieval:
  retrieval[key].sort(key=lambda x:x[1], reverse=True)

print(retrieval)

100
{'tc_7': [('Asmara.txt', 0.9986739158630371), ('Asmara_International_Airport.txt', 0.9986680746078491), ('International_airport.txt', 0.9986637830734253), ('Queen_Alia_International_Airport.txt', 0.9986587762832642), ('O._R._Tambo_International_Airport.txt', 0.9986554384231567), ('Airport.txt', 0.9986535310745239), ('Eritrea.txt', 0.9986521601676941), ('Hartsfield–Jackson_Atlanta_International_Airport.txt', 0.9986442923545837), ('Simón_Bolívar_International_Airport_(Venezuela).txt', 0.9986432194709778), ('Keflavík_International_Airport.txt', 0.9986363053321838), ('Kotoka_International_Airport.txt', 0.9986328482627869), ('Abu_Dhabi_International_Airport.txt', 0.9986327290534973), ('Benina_International_Airport.txt', 0.9986318945884705), ('Ben_Gurion_Airport.txt', 0.9986251592636108), ('Burgas_Airport.txt', 0.9986118078231812), ('Zürich_Airport.txt', 0.9986110925674438), ('Malta_International_Airport.txt', 0.9986044764518738), ('Ted_Stevens_Anchorage_International_Airport.txt', 0.998

In [ ]:
save_as_pickle(retrieval, output_dir)

In [ ]:
del retrieval
del model
del preds_array

## Second Cycle: 5000 - 15000 Roberta - Random False Positives

In [ ]:
model = RobertaForSequenceClassification.from_pretrained(model_path_roberta_second_randhard_5000_15000_wd_2e)
output_dir = bm25_rr_100_roberta_5000_15000_wd_2_rh

In [ ]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings, boundaries=None):
    if boundaries is not None:
      self.encodings = {key: val[boundaries[0]: boundaries[1]] for key, val in encodings.items()}
    else:
      self.encodings = encodings

  def __getitem__(self, idx):
      item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
      return item

  def __len__(self):
      return len(self.encodings['input_ids'])

trainer = Trainer(
    model = model
)

def get_preds(dataset):
  prediction_output = trainer.predict(dataset)
  predictions = prediction_output.predictions
  preds = torch.tensor(predictions)
  res = F.softmax(preds, dim = 1)
  results = res[:,1]
  del predictions
  del prediction_output
  del preds
  del res
  torch.cuda.empty_cache()
  return results

In [ ]:
sub_dataset = Dataset(encodings)
preds_array = get_preds(sub_dataset)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


In [ ]:
print(preds_array)
print(torch.max(preds_array))
print(torch.min(preds_array))

tensor([0.9882, 0.9884, 0.9731,  ..., 0.0033, 0.0038, 0.0034])
tensor(0.9888)
tensor(0.0031)


In [ ]:
## connect predictions w/ query_id and docs
i= 0
retrieval = {}
qids = []
for qa_pair in preprocessed_qa_wikipedia_test_without_answers[:100]:
  query_id = qa_pair['QuestionId']
  for d_id in test_top_50_bm25[query_id]:
    if query_id in qids:
      retrieval[query_id].append((d_id, float(preds_array[i])))
    else: 
      retrieval[query_id]= [(d_id, float(preds_array[i]))]
      qids.append(query_id)
    i +=1
    

print(len(retrieval))

for key in retrieval:
  retrieval[key].sort(key=lambda x:x[1], reverse=True)

print(retrieval)

100
{'tc_7': [('Asmara.txt', 0.9883749485015869), ('Asmara_International_Airport.txt', 0.9881917834281921), ('International_airport.txt', 0.9840713739395142), ('Eritrea.txt', 0.9730979204177856), ('Airport.txt', 0.9705140590667725), ('Queen_Alia_International_Airport.txt', 0.9347216486930847), ('Malta_International_Airport.txt', 0.007529357448220253), ('Abu_Dhabi_International_Airport.txt', 0.00690794549882412), ('O._R._Tambo_International_Airport.txt', 0.005719476845115423), ('Keflavík_International_Airport.txt', 0.004882679786533117), ('Benina_International_Airport.txt', 0.00483199767768383), ('Ben_Gurion_Airport.txt', 0.0047182729467749596), ('Simón_Bolívar_International_Airport_(Venezuela).txt', 0.004314542748034), ('East_Africa.txt', 0.004115324933081865), ('Zürich_Airport.txt', 0.00409332662820816), ('Kotoka_International_Airport.txt', 0.004070845432579517), ('Amílcar_Cabral_International_Airport.txt', 0.003943733870983124), ('Transport_in_Germany.txt', 0.0037019168958067894), ('

In [ ]:
save_as_pickle(retrieval, output_dir)